# extract NER Training Samples from csae8092/busoni-data-public

In [1]:
from github import Github
import json
import os
import acdh_tei_pyutils
from acdh_tei_pyutils.tei import TeiReader

In [2]:
clean_markup = os.path.join(acdh_tei_pyutils.__path__[0], 'files', 'clean_markup.xsl')

In [3]:
# secret.json = {"login_or_token": "username", "password": "password"}

In [4]:
with open('secret.json', 'r') as f:
    login = json.load(f)

In [5]:
g = Github(**login)

In [ ]:
repo = g.get_repo('csae8092/busoni-data-public')

In [ ]:
contents = repo.get_contents("text/letters")

In [ ]:
with open("sample.jsonl", "w") as a_file:
    contents = repo.get_contents("text/letters")
    while contents:
        file_content = contents.pop(0)
        if file_content.type == "dir":
            contents.extend(repo.get_contents(file_content.path))
        else:
            dl_url = file_content._rawData.get('download_url')
            if not 'description' in dl_url:
                print(dl_url)
                doc = TeiReader(xml=dl_url, xsl=clean_markup)
                parent_node = doc.any_xpath('.//tei:body')[0]
                try:
                    ne_list = doc.extract_ne_offsets(
                        parent_nodes='.//tei:body//tei:p',
                        ne_xpath=".//*[contains(name(), 'Name') or name()='date' or name()='time']"
                    )
                except:
                    continue
                for y in ne_list:
                    item = {
                        "text": y[0],
                        "entities": y[1]['entities']
                    }
                    a_file.write("\n")
                    a_file.write(json.dumps(item, ensure_ascii=False))
                    